**This project is due Wednesday, December 03, 2025 at 11:59 pm. Please plan ahead and submit your work on time.**

<center><h1>Grad Project #3</h1></center>

In this project, continuing with the "search and rescue" domain, we will consider the problem of inferring the location of the robot as it moves around. Specifically, you will:

* Implement a particle filter for the localization problem, to allow the robot to estimate its position. 
* Implement a particle filter for the simultaneous localization and mapping (SLAM) problem, to allow the robot *both* to estimate its position and the map of its environment.  
* Run simulations to test the performance of your algorithms.

Your project will consist of implementing a particle filter, and also performing experiments and analysis on the different implementations. Your project submission will consist of the code you implement, a PDF of your experiments, and the movies we've asked you to generate.

Your project grade will be based on the autograded tests, report (with figures), animation, and notebook as a whole. 

0. [Credit for Contributors (required)](#contributors)
1. [Noise Sampling (12 points)](#sampling)
    1. [Sampling Warmups (2 points)](#warmups)
        1. [Warmup 1 (1 point)](#warmup1)
        2. [Warmup 2 (1 point)](#warmup2)
    2. [Adding Motion Noise (10 points)](#motion_noise)
        1. [Experiment 1: Sampling from Multivariate Gaussian (5 points)](#multivariate)
        2. [Experiment 2: Motion Noise (5 points)](#noise)
2. [Particle Filter Localization (48 points)](#pf_localization)
    1. [Motion Model (28 points)](#motion_model)
        1. [Motion Model Implementation (3 points)](#motion_imp)
        2. [Experiment 3: Noisy Motion (5 points)](#exp3)
        3. [Experiment 4: Less Noisy Motion (5 points)](#exp4)
        4. [Experiment 5: Finer Motion (5 points)](#exp5)
        5. [Experiment 6: Adjusting Noise for Finer Motion (5 points)](#exp6)
        6. [Experiment 7: Unbalanced Noise (5 points)](#exp7)
    2. [Localization: Computing the Importance Weights (10 points)](#localization_importance)
    3. [Localization: Particle Filter Update (5 points)](#localization_pfupdate)
    4. [Evaluating Particle Filter-based Localization (5 points)](#evaluating_pf_localization)
        1. [Experiment 8: Running the Localization Particle Filter (5 points)](#exp8)
3. [Particle Filter SLAM (40 points)](#pf_slam)
    1. [SLAM: Update by Motion Model (5 points)](#slam_motion_model)
    2. [SLAM: Tracking New Landmarks (10 points)](#slam_tracking)
    3. [SLAM: Computing the Importance Weight (10 points)](#slam_importance)
    4. [SLAM: Particle Filter Update (5 points)](#slam_pf_update)
    5. [Evaluating Particle Filter-based SLAM (10 points)](#evaluating_pf_slam)
        1. [Experiment 9: Running the SLAM Particle Filter (5 points)](#exp9)
        2. [Experiment 10: More Fun with the SLAM Particle Filter (5 points)](#exp10)

# <a id="contributors"></a> 0. Credit for Contributors

List the various students, lecture notes, or online resouces that helped you complete this project:

Ex: I worked with Bob on Sampling.

<div class="alert alert-info">
Write your answer in the cell below this one.
</div>

--> *(double click on this cell to delete this text and type your answer here)*

# Imports and Utilities

These are import and utility functions, and also scaffolding of functions that we have provided for you for the project.

In [ ]:
# Setup matplotlib animation
import matplotlib
matplotlib.rc('animation', html='jshtml')
import random; import numpy.random; random.seed(0); numpy.random.seed(0);

%load_ext autoreload
%autoreload 2
from principles_of_autonomy.grader import Grader
from principles_of_autonomy.notebook_tests.proj_3 import TestProj3

In [ ]:
from typing import Optional


def implementation_for(cls, method_name: Optional[str] = None):
    """Helper to provide a concrete implementation for a class method.

    This utility allows one to define methods outside of a class defintion.
    Thus allowing one to provide the implementations of a class "incrementally".
    This utility is useful for gradually submitting class method implementations
    to catsoop.

    It is intended to be used as a decorator.

    Args:
        cls: a Python class on which we are implementing the method.
        method_name: an optional method name to indicate which method we are
            implementing. If not provided, infer the method name by using the
            decorated function's name.

    Example:
        >>> class A:
        ...   def foo(self):
        ...     raise NotImplementedError()

        >>> @implementation_for(A)
        ... def foo(self):
        ...   print("implemented foo!")

        >>> A().foo()
        implemented foo!
    """

    def decorator(meth: Callable) -> Callable:
        mname = method_name or meth.__name__
        setattr(cls, mname, meth)
        return meth

    return decorator



from typing import (Callable, Iterable, List, Sequence, Optional, Dict)

from abc import abstractmethod, ABCMeta
import itertools
import math
import random
import dataclasses
from collections import namedtuple
import copy

import numpy as np

import scipy.stats
import scipy.special

# Basic data structures.

# x, y coordinate and the rotation of the robot.
Pose = namedtuple("Pose", ["x", "y", "theta"])
# unique id (str) and x, y coordinate of the landmark.
Landmark = namedtuple("Landmark", ["id", "x", "y"])
# r --- distance to the landmark; b --- angle to the landmark, computed by arctan2.
Measurement = namedtuple("Measurement", ["landmark_id", "r", "b"])
# delta_p --- distance to move; delta_theta --- rotation of the robot
Command = namedtuple("Command", ["delta_p", "delta_theta"])


def grid_of_landmarks(
        x_range: Iterable[float] = range(-4, 20, 4),
        y_range: Iterable[float] = range(-4, 20, 4),
) -> Sequence[Landmark]:
    """Constructs a grid of landmarks from a catesion product of x and y coordinates."""
    return tuple(
        Landmark(f"landmark-{i}", *loc)
        for i, loc in enumerate(itertools.product(x_range, y_range)))


class Inference(metaclass=ABCMeta):
    """Interface for an inference algorithm (localization or SLAM variants)."""

    @abstractmethod
    def estimated_pose(self) -> Pose:
        """Returns the current estimation of the robot's Pose."""
        ...

    @abstractmethod
    def estimated_landmarks(self) -> Sequence[Landmark]:
        """Returns the current estimation of the landmarks."""
        ...

    @abstractmethod
    def init(self, init_state: Pose) -> None:
        """Initialize the inference with an initial pose."""
        ...

    @abstractmethod
    def update(self, measurements: Sequence[Measurement]) -> None:
        """An update step of the inference, called after every simulation step."""
        ...

    @abstractmethod
    def plot_state(self, ax) -> None:
        """Helper to visualize the internal state of the inference algorithm."""
        ...


def normalize_angles(angles: np.array) -> np.ndarray:
    """Given an array of angles in radians, element-wise normalize the angles to the range [-pi, pi].

    Args:
        angles: array of any shape.

    Returns:
        normalized angles, array of same shape as the input.
    """
    return np.arctan2(np.sin(angles), np.cos(angles))


def circular_mean(angles: np.array, axis=None) -> float:
    """Given an array of angles in radians, find the [circular mean](https://en.wikipedia.org/wiki/Circular_mean).

    Args:
        angles: array of any shape.
        axis: axis or axes along which the means are computed. The default is to compute the mean of the flattened array.

    Returns:
        mean of the angles, normalized to [-pi, pi].
    """
    return np.arctan2(np.sum(np.sin(angles), axis=axis),
                      np.sum(np.cos(angles), axis=axis))


@dataclasses.dataclass(frozen=True)
class SimulationResult:
    """A helper class to hold the results of running a simulation."""
    sim: 'Simulator'  # Simulator that generated this result
    infer: Optional[Inference]  # Inference procedure that generated this result
    true_poses: Sequence[Pose]
    estimated_poses: Optional[Sequence[Pose]] = None
    # History snapshots for the Inference, for animation purpose
    snapshots: Optional[Sequence[Inference]] = None


@dataclasses.dataclass(frozen=False)
class Simulator:
    """A simulator for a point robot in R^2.

    The simulator simulates the motion and sensor noises of the robot by
    sampling from 2D Gaussians.
    The simulation has a list of landmarks, each with a unique identifier.
    The sensor of the robot can sense, for each landmark in the robot's sensing
    range, a noise-corrupted distance and bearing to that landmarks (together
    with the identifier that identifies the landmark).
    """

    # Initial simulator state
    init_state: Pose = Pose(0, 0, 0)

    # 2x2 covariance matrix of the motion noise.
    motion_noise_covariance: np.ndarray = np.diag([1e-3, np.deg2rad(3)**2])

    # 2x2 covariance matrix of the sensor noise.
    sensor_noise_covariance: np.ndarray = np.diag([1, np.deg2rad(5)**2])

    # A set of landmarks
    landmarks: Sequence[Landmark] = grid_of_landmarks()

    # Robot can only sense landmarks within this range
    max_sensing_range: float = np.inf

    # Current state of the robot
    state: Pose = dataclasses.field(init=False, default=None)

    # Random seed control
    rng: np.random.Generator = dataclasses.field(init=False, default=None)
    # Using the same seed will induce the same trajectory
    seed: dataclasses.InitVar[int] = 0

    def __post_init__(self, seed: int):
        assert (len({l.id for l in self.landmarks
                    }) == len(self.landmarks)), "Landmark must have unique IDs"
        self.rng = np.random.default_rng(seed)

    def init(self) -> None:
        self.state = self.init_state

    def simulate_motion(self, command: Command) -> None:
        """Simulate robot motion. This function updates the value of
        self.state.

        Args:
          command: a Command tuple containing fields delta_p(float), the
          distance of the movement, and delta_theta(float), the rotation of
          the movement.
        """
        noise = self.rng.multivariate_normal(np.zeros(2),
                                             self.motion_noise_covariance)
        x = (self.state.x + math.cos(self.state.theta) *
             (command.delta_p + noise[0]))
        y = (self.state.y + math.sin(self.state.theta) *
             (command.delta_p + noise[0]))
        theta = self.state.theta + command.delta_theta + noise[1]
        self.state = Pose(x, y, theta)

    def simulate_sensing(self) -> Sequence[Measurement]:
        """Simulate the robot sensing process. This function returns a list of
        measurements, in the same order as the self.landmarks list.
        Specifically, it measures the distance and the angle w.r.t. each
        landmark, and add a Gaussian noise on the measurement.

        Returns:
          measurements: a list of measurements.
        """
        measurements = []
        # Simulate that the measurements' order is random
        for landmark in sorted(self.landmarks, key=lambda _: random.random()):
            r = math.hypot(landmark.x - self.state.x, landmark.y - self.state.y)

            if r >= self.max_sensing_range:
                # Can't sense this landmark, skip
                continue

            b = math.atan2(landmark.y - self.state.y,
                           landmark.x - self.state.x) - self.state.theta
            # Normalize angle to -pi to pi
            b = math.atan2(math.sin(b), math.cos(b))

            noise = self.rng.multivariate_normal(np.zeros(2),
                                                 self.sensor_noise_covariance)

            measurements.append(
                Measurement(landmark.id, r + noise[0], b + noise[1]))

        return measurements

    def run(self,
            commands: Iterable[Command],
            infer: Optional[Inference] = None,
            snapshot_every_n: Optional[int] = 0) -> SimulationResult:
        """Run the simulation with a sequence of commands. Optionally, execute
        an inference algorithm along way with the simulation.

        Args:
            commands: an interable of commands.
            infer: the inference algorithm to run the simulation with.
            snapshot_every_n: snapshot the inference algorithm every n steps.
                if <= 0, don't take any snapshot.

        Returns:
            SimulationResult
        """
        true_poses, est_poses, snapshots = [], None, None

        self.init()

        i = 0
        true_poses.append(self.state)
        if infer:
            est_poses, snapshots = [], []
            infer.init(self.init_state)
            est_poses.append(infer.estimated_pose())
            if snapshot_every_n > 0 and i % snapshot_every_n == 0:
                snapshots.append(copy.deepcopy(infer))

        for command in commands:
            i += 1
            # Simulate motion
            self.simulate_motion(command)
            true_poses.append(self.state)
            if infer:
                # Simulate measurement
                measurements = self.simulate_sensing()
                infer.update(command, measurements)
                est_poses.append(infer.estimated_pose())
                if snapshot_every_n > 0 and i % snapshot_every_n == 0:
                    snapshots.append(copy.deepcopy(infer))

        return SimulationResult(self, copy.deepcopy(infer), true_poses,
                                est_poses, snapshots)


def drive_in_a_square_commands() -> Iterable[Command]:
    """Commands to simulate the robot's movement in a square."""
    for _ in range(4):
        for i in range(100):
            yield Command(.1, 0)
        yield Command(0, np.pi / 2)


def patrol_commands() -> Iterable[Command]:
    """Commands that walks back and forth horizontally."""
    for i in range(10):
        yield Command(0.5, 0)
    yield Command(0, np.pi)





def run_inference(sim: Simulator,
                  commands: Sequence[Command] = tuple(
                      drive_in_a_square_commands()),
                  mode: str = "localization",
                  num_particles: int = 100,
                  snapshot_every_n=0) -> SimulationResult:
    """Utility to run inference on a simulator with a particular inference mode.

    Args:
        sim: the simulator instance to run the simulation.
        commands: see `Simulator.run`.
        mode: specified the mode of the inference; one of "localization", "SLAM" or "RBSLAM".
            You must implement the corresponding inference algorithm in order
            to perform inference in any mode.
        num_particles: number of particles of the particle filter.
        snapshot_every_n: see `Simulator.run`.

    Returns:
        A simulation result returned by `Simulator.run`.
    """

    mode = mode.lower()
    try:
        if mode == "localization":
            infer = Localization(
                motion_noise_covariance=sim.motion_noise_covariance,
                sensor_noise_covariance=sim.sensor_noise_covariance,
                landmarks=sim.landmarks,
                num_particles=num_particles,
            )
        elif mode == "slam":
            infer = SLAM(
                motion_noise_covariance=sim.motion_noise_covariance,
                sensor_noise_covariance=sim.sensor_noise_covariance,
                num_particles=num_particles,
            )
        elif mode == "rbslam":
            infer = RBSLAM(
                motion_noise_covariance=sim.motion_noise_covariance,
                sensor_noise_covariance=sim.sensor_noise_covariance,
                num_particles=num_particles,
            )
        else:
            raise ValueError(f"Unrecoginized inference: {mode}")
    except NotImplementedError:
        raise NotImplementedError(
            f"You must completely implement the {infer.__class__} class to "
            f"run simulation under {mode} mode")

    return sim.run(commands, infer=infer, snapshot_every_n=snapshot_every_n)


def plot_samples(x, y, fov=((-6, -6), (20, 20)), ax=None, show=True):
    """Plots a list of samples of x, y coordinates as input and scatter plot the samples.

    Args:
        x: array of shape (N, ), the x-coordinates of the pose component of the particles.
        y: array of shape (N, ), the y-coordinates of the pose component of the particles.
        fov: a rectangular field of view. The function plots within this
            rectanglular fov.
        show: whether to display the plot immediately.
    """
    import matplotlib.pyplot as plt
    if ax is None:
        fig, ax = plt.subplots()

    ax.axis([fov[0][0], fov[1][0], fov[0][1], fov[1][1]])
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_aspect('equal', 'box')

    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.plot(x, y, 'g+')

    if show:
        plt.show()

    return ax


def plot_trajectories(true_poses: Sequence[Pose],
                      estimated_poses: Sequence[Pose],
                      landmarks: Optional[Sequence[Landmark]] = None,
                      ax: Optional['matplotlib.axes.Axes'] = None,
                      fov=((-6, -6), (20, 20)),
                      show=True):
    """Plot true and estimated poses on the same plot by overlaying them.
    It ignores the pose.theta and only plots the location.

    Args:
        true_poses: a sequence of true poses.
        estimated_poses: a sequence of estimated poses.
        landmarks: optinally plot the location of true landmarks.
        ax: an optional matplotlib Axes on which to plot the trajectories.
        fov: a rectangular field of view. The function plots within this
            rectanglular fov.
        show: whether to display the plot immediately.
    """
    import matplotlib.pyplot as plt
    if ax is None:
        fig, ax = plt.subplots()

    ax.axis([fov[0][0], fov[1][0], fov[0][1], fov[1][1]])
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_aspect('equal', 'box')

    if landmarks:
        ax.scatter([l.x for l in landmarks], [l.y for l in landmarks],
                   marker="o",
                   color="black")

    ax.plot([p.x for p in true_poses], [p.y for p in true_poses],
            color=(0, 0, 1, 0.5))
    ax.plot([p.x for p in estimated_poses], [p.y for p in estimated_poses],
            color=(1, 0, 0, 0.5))

    if show:
        plt.show()
    return ax


def plot_simulation_result(result: SimulationResult,
                           ax: Optional['matplotlib.axes.Axes'] = None,
                           fov=((-6, -6), (20, 20)),
                           title=None,
                           show=True):
    """Plot the inference result (the state at last step of the simulation result).

    Args:
        result: result of a simulation.
        ax: an optional matplotlin Axes to draw the result.
        fov: see `plot_trajectories`.
        show: whether to display the plot immediately.
    """
    import matplotlib.pyplot as plt
    if ax is None:
        fig, ax = plt.subplots()

    if title:
        ax.set_title(title)

    plot_trajectories(result.true_poses,
                      result.estimated_poses,
                      landmarks=result.sim.landmarks,
                      ax=ax,
                      fov=fov,
                      show=False)

    result.infer.plot_state(ax)

    if show:
        plt.show()

    return ax


def render_animation(
    result: SimulationResult,
    fov=((-6, -6), (20, 20)),
    save_file: Optional[str] = None,
) -> 'matplotlib.animation.FuncAnimation':
    """Render simulation as a matplotlib animation.

    Args:
        result: result of a simulation.
        fov: see `plot_trajectories`.
        save_file: an optional string file name to save the animation.

    Returns:
        an instance of `matplotlib.animation.FuncAnimation`. You can do further
        processing on the returned animation, display it, or save it
        as an mp4 file.
    """
    if result.snapshots is None or len(result.snapshots)==0:
        raise ValueError("SimulationResult does not have any snapshots. "
                         "Please re-run simulation with snapshot_every_n > 0.")

    import matplotlib.animation as animation
    import matplotlib.pyplot as plt

    fig, ax = plt.subplots()

    snapshot_every_n = len(result.true_poses) // len(result.snapshots)
    true_poses = result.true_poses[::snapshot_every_n]
    estimated_poses = result.estimated_poses[::snapshot_every_n]

    def render_frame(i):
        ax.clear()
        ax.set_title(f"Step {i}")
        plot_trajectories(true_poses[:i + 1],
                          estimated_poses[:i + 1],
                          landmarks=result.sim.landmarks,
                          ax=ax,
                          fov=fov,
                          show=False)
        result.snapshots[i].plot_state(ax)
        return ax

    anim = animation.FuncAnimation(fig, render_frame, len(result.snapshots))
    if save_file:
        anim.save(f"{save_file}.mp4", fps=60)
    return anim

# <a id="sampling"></a> 1. Noise Sampling (15 points)

## <a id="warmups"></a> 1A. Sampling Warmups (2 points)

### <a id="warmup1"></a> Warmup 1 (1 point)

Let's make sure we know how to sample from a set of values based on their weights.

Please write a function that:
- Takes in a list of samples and associated weights.
- Re-samples from the list of samples with replacement according the
likelihood of each sample given by the list of weights.

_Hint:_ You should use the function `numpy.random.choice`.


For reference, our solution is **1** line(s) of code.

In [ ]:
def warmup_1(samples: np.ndarray, weights: np.ndarray,
             nr_samples: int, seed=None) -> np.ndarray:
    """Draw N samples from the input list based on their weights.

    Args:
      samples: a numpy array of shape (n_samples, ), a list of samples.
      weights: a numpy float array of shape (n_samples, ), indicating
        the weights for individual samples.
      nr_samples: an integer, indicating the number of samples to draw.
      seed: optional, for reproducible results in testing

    Returns:
      resampled_samples: the return numpy array, containing nr_samples
        samples drawn from the input list, based on the weights with replacement.
    """
    # For reproducible results in testing
    if seed is not None:
        np.random.seed(seed)
    raise NotImplementedError() 

In [ ]:
# Test 1
Grader.run_single_test_inline(TestProj3, "test_01_warmup_1", locals())

### <a id="warmup2"></a> Warmup 2 (1 point)

We are going to be sampling from Gaussian distributions in this project. While we have not talked about continuous random variables and continuous probability densities much during the lectures, we can use the ideas from Monte Carlo inference techniques to handle continuous variables quite easily. 

First, let us check that we know how to sample from a multivariate Gaussian.
Please write a function that takes in a mean vector, a covariance matrix and a integer number
of samples and returns a list of the requested number of samples drawn from a Gaussian with the given mean and covariance.

_Hint:_ You should use the function `numpy.random.multivariate_normal`.

For reference, our solution is **1** line(s) of code.

In [ ]:
def warmup_2(mean: np.ndarray, covariance: np.ndarray,
             num_particles: int, seed=None) -> np.ndarray:
    """Sample from a multi-variate Gaussian distribution.

    Args:
      mean: a numpy array of shape (N, ).
      covariance: a numpy array of shape (N, N).
      num_particles: an integer.
      seed: optional, for reproducible results in testing

    Returns:
      sample: a numpy vector with length num_particles.
    """
    # For reproducible results in testing
    if seed is not None:
        np.random.seed(seed)
    raise NotImplementedError() 

In [ ]:
# Test 2
Grader.run_single_test_inline(TestProj3, "test_02_warmup_2", locals())

## <a id="motion_noise"></a> 1B. Adding Motion Noise (10 points)

### <a id="multivariate"></a> Experiment 1: Sampling from Multivariate Gaussian (5 points)

Please generate a 2D scatter plot of 1000 samples drawn from a 2D Gaussian with `mean = [0, 0]` and `covariance = np.diag([0.05, 0.01])`, using the function you just coded: `warmup_2`. Consider using the helper function provided: `plot_samples`.

<b>Submission Material 1a:</b> In your submitted pdf, please include the generated figure. Name this Figure 1a.

Let's get some intuition about geometry. Please generate a second 2D scatter plot of 1000 samples drawn from a 2D Gaussian distribution. You should pick the mean and covariance so that the figure shows an ellipse with a $45$-degree angle with the x-y axes. Specifically, try to modify the mean and covariance to rotate figure 1a by 45-degrees counter-clockwise.


<b>Submission Material 1b:</b> In your submitted pdf, please include the generated figure. Name this Figure 1b.

In [ ]:
# Submission Material 1a and 1b
raise NotImplementedError()

### <a id="noise"></a> Experiment 2: Motion Noise (5 points)

We have provided a simulator class for you, that simulates the noisy motion
and sensing of the robot. Let's begin by ignoring any observations and just trusting that the robot goes where we have commanded. We'll look at the relationship between commands and actual trajectories.

We have provided a `Simulator` class for you. It simulates the noisy motion and sensing of the robot. Let's begin by exploring what happens if we don't do any inference, and assume we always know where the robot is.

We have provided a method `Simulator.run` that simulates the motion of the robot given a sequence of commands. For now, you may ignore the `infer` parameter. If you call this method, you will get a `SimulationResult`. Here, `SimulationResult.true_poses` holds a list of poses representing the ''true'' trajectory of the robot. You can ignore the rest of the fields in `SimulationResult` for now.

Please call `run` twice, passing in `drive_in_a_square_commands()` as the commands.
- For the first time, run on a simulator with `motion_noise_covariance=zeros((2, 2))`
- For the second time, run on a simulator with `motion_noise_covariance=np.diag([1e-3, np.deg2rad(3)**2])`. 

The `drive_in_a_square_commands()` generates a sequence of commands that attempt to drive the robot in a square motion.
However, you should see a significant difference between the trajectories without noise, which should be a perfect square, versus with noise. 

**Hint**: You may use the helper function `plot_trajectories` to generate the trajectory plot.

<b>Submission Material 2:</b> In your submitted pdf, please include a single figure which has both experiment results overlapped on each other. Name this Figure 2.
If we knew the noise on each step, the noise wouldn't be a problem. We could just correct for it as we steer the robot, but the problem is we don't the noise --- it's just the random perturbations added by nature. Therefore, we have to use the sensors to know where the robot is and correct for the "drift" in the robot motion. **That** is why we perform inference.

Now we've convinced ourselves of the need for inference, let us implement the two functions we need for sampling-based inference.

In [ ]:
# Submission Material 2
raise NotImplementedError()

# <a id="pf_localization"></a> 2. Particle Filter Localization (48 points)

Now, let’s assume that there are some landmarks (e.g. radio beacons or visible mountains in the distance) with known absolute locations. The robot can observe measurements of its relative position to these landmarks. This problem becomes an HMM, in which the hidden state is the robot’s position and orientation (called a "pose" for short), the noisy steering we saw above is the transition model, and the landmark measurements are the observations. We are interested in performing filtering to obtain a distribution over the robot’s pose at each time step. 

We could discretise the robot position, and perform HMM inference as we saw in class, but discretising the states is going to lead to a very large discrete state space that will be hard to perform inference efficiently in. If you have taken a robotics course or state estimation course elsewhere, you might be familiar with the Kalman filter as a way to estimate the state of a linear system with Gaussian noise, but as we will see, the transition dynamics of our robot are substantially non-linear and a Kalman filter will not perform well.

Instead, we will use a particular form of sampling over HMMs known as the particle filter, which we discussed briefly at the very end of the HMM lecture. 

### Utilities

Please have a look at the following utility functions for localization update by motion model:

In [ ]:
from functools import cached_property

def multivariate_normal_logpdf(xs: np.ndarray, means: np.ndarray,
                               covs: np.ndarray) -> np.ndarray:
    """Batch version of `scipy.stats.multivariate_normal.logpdf`.

    Compute multivariate normal's log probability density function, with batch
    inputs and distributions.
    Modified from https://gregorygundersen.com/blog/2020/12/12/group-multivariate-normal-pdf/.

    This function uses the following broadcasting logic:
    - First, `means` and `covs` are broadcasted together, which determines `dist_batch`,
        the number of individual multivariate normal distributions.
    - Then, broadcast the rightmost dimensions of `xs` to `means`. The rest of the dimensions
    of `xs` becomes the `input_batch` dimension.

    Args:
        xs: array of shape (...input_batch,  ...dist_batch, dist_dim)
        means: array of shape (...dist_batch, dist_dim)
        covs: array of shape (...dist_batch, dist_dim, dist_dim)

    Returns:
        array of shape (...input_batch, ...dist_batch, dist_dim)


    Below we describe some uses cases, with different input shapes and the result shape.

    1.  xs: (100, 2), means: (2,),  covs: (2, 2), result: (100, 2)
        Explanation:
            - dist_dim = 2 i.e., Gaussian is 2D
            - `input_batch=(100,)` and `dist_batch=()`
            - returns logpdf of 100 inputs, evaluated by the same mean and covariance.

    2.  xs: (100, 2), means: (2,),  covs: (100, 2, 2), result: (100, 2)
        Explanation:
            - dist_dim = 2 i.e., Gaussian is 2D
            - `input_batch=()` and `dist_batch=(100,)`
            - returns logpdf of 100 inputs, evaluated by the same mean but with 100 different covariances.

    3.  xs: (100, 10, 2), means: (2,),  covs: (100, 10, 2, 2), result: (100, 10, 2)
        Explanation:
            - dist_dim = 2 i.e., Gaussian is 2D
            - `input_batch=()` and `dist_batch=(100, 10)`
            - returns logpdf of inputs of shape (100, 10), evaluated by the same mean each with a different covariance.
    """
    dist_dim = xs.shape[-1]
    dist_batch_shape = np.broadcast_shapes(means.shape[:-1], covs.shape[:-2])
    means = np.broadcast_to(means, dist_batch_shape + (dist_dim,))
    covs = np.broadcast_to(covs, dist_batch_shape + (dist_dim, dist_dim))
    xs = np.broadcast_to(xs, xs.shape[:-len(means.shape)] + means.shape)

    vals, vecs = np.linalg.eigh(covs)
    logdets = np.sum(np.log(vals), axis=-1)
    valsinvs = 1. / vals
    Us = vecs * np.sqrt(valsinvs)[..., np.newaxis, :]
    devs = xs - means
    devUs = np.einsum("...i,...ij->...j", devs, Us)
    mahas = np.sum(np.square(devUs), axis=-1)
    log2pi = np.log(2 * np.pi)
    out = -0.5 * (dist_dim * log2pi + mahas + logdets)
    return out


@dataclasses.dataclass(frozen=True)
class LocalizationParticles:
    """A batch of particles."""

    x: np.ndarray  # shape (N, )
    y: np.ndarray  # shape (N, )
    theta: np.ndarray  # shape (N, )

    def __post_init__(self):
        assert self.x.shape == self.y.shape == self.theta.shape
        assert len(self.x.shape) == 1
        # Always normalize the angles
        object.__setattr__(self, "theta", normalize_angles(self.theta))

    def __len__(self) -> int:
        """The number of particles."""
        return self.x.shape[0]


@dataclasses.dataclass
class Localization(Inference):
    """Localization Inference by particle filtering."""

    # A 2x2 array for the motion covariance.
    motion_noise_covariance: np.ndarray

    # A 2x2 numpy array for the sensor noise for the
    # measurements of range and bearing to the landmarks
    sensor_noise_covariance: np.ndarray

    # In localization, we assume that we have access to the set of landmarks.
    landmarks: List[Landmark] = dataclasses.field(default_factory=list)

    num_particles: int = 10

    particles: LocalizationParticles = None

    def __post_init__(self):
        if self.particles:
            assert len(self.particles) == self.num_particles

    @cached_property
    def landmarks_id_map(self) -> Dict[str, Landmark]:
        """Map from unique landmark id to the Landmark instance."""
        return {l.id: l for l in self.landmarks}

    def estimated_pose(self) -> Pose:
        return Pose(np.mean(self.particles.x), np.mean(self.particles.y),
                    circular_mean(self.particles.theta))

    def estimated_landmarks(self) -> Sequence[Landmark]:
        # we know the landmarks so no estimation required!
        return self.landmarks

    def init(self, init_state: Pose) -> None:
        self.particles = LocalizationParticles(
            np.full(self.num_particles, init_state.x, dtype=np.float64),
            np.full(self.num_particles, init_state.y, dtype=np.float64),
            np.full(self.num_particles, init_state.theta, dtype=np.float64),
        )

    def plot_state(self, ax: 'matplotlib.axes.Axes') -> None:
        ax.quiver(
            self.particles.x,
            self.particles.y,
            np.cos(self.particles.theta),
            np.sin(self.particles.theta),
            angles="xy",
            scale=100,
            color=(1.0, 0, 0, 0.4),
        )

    # We incrementally implement methods below using the `implementation_for` helper
    def motion_model(self, command: Command):
        raise NotImplementedError()

    def compute_weights(self,
                        measurements: Sequence[Measurement]) -> np.ndarray:
        raise NotImplementedError()

    def update(self, command: Command, measurements: Sequence[Measurement]):
        raise NotImplementedError()


def plot_particles(samples: LocalizationParticles, fov=((-6, -6), (20, 20))):
    """matplotlib helper function. It takes a list of samples as input and
    scatter plot the samples.

    Args:
        particles: a set of particles for localization. This function only plots
            the first two components: x and y.
        fov: a rectangular field of view. The function plots within this
            rectanglular fov.
    """
    import matplotlib.pyplot as plt
    fig, ax = plt.subplots()

    ax.axis([fov[0][0], fov[1][0], fov[0][1], fov[1][1]])

    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.plot(samples.x, samples.y, 'g+')
    plt.show()

## <a id="motion_model"></a> 2A. Motion Model (25 points)

### <a id="motion_imp"></a> Motion Model Implementation (3 points)

Let us now start by building a motion model for a
simple ground robot that moves in straight lines, and performs point turns.

A common motion model for such a robot assumes that the robot has a pose given
by $[x_t, y_t, \theta_t]$ at time $t$. The robot takes a command $[\Delta p_t,
\Delta \theta_t]$ which modifies the robot's pose according to the following:

$$
\begin{bmatrix} x_{t+1} \\ y_{t+1} \\ \theta_{t+1} \end{bmatrix} =
\begin{bmatrix} x_t + \cos(\theta_t) (\Delta p_t + \omega_t) \\
y_t + \sin(\theta_t)(\Delta p_t + \omega_t) \\
\theta_t + \Delta \theta_t + \nu_t
\end{bmatrix}
$$

where $[\omega_t, \nu_t]$ is Gaussian error injected at each time
step, distributed as $\omega_t \sim \mathcal{N}(0, \sigma^2_p)$, and
$\nu_t \sim \mathcal{N}(0, \sigma^2_\theta)$. The units of $\Delta  p_t$ is in
metres and $\Delta \theta_t$ in radians.

Let us assume that the robot's pose at each time step is a random variable,
represented as a set of samples.

Please implement a function that updates the set of sampled poses (particles)
drawn from prior pose random variable $\mathbf{X}_t$
using a control $[\Delta p_t, \Delta \theta_t]$ to a set of samples drawn from the posterior $X_{t+1}$.

**Note**
In this project, we will use a helper decorator `implementation_for` to implement
class methods outside of the class.
The decorator `@implementation_for(Localization)` simply inserts the declared
function as a method of the Localization class.

**Hint**: Use `np.random.multivariate_normal` to generate the noise.


For reference, our solution is **8** line(s) of code.

In [ ]:
@implementation_for(Localization)
def motion_model(self: Localization, command: Command) -> None:
    """A motion model that simulates a one-step movement of the robot, and
    updates to the new samples.

    Args:
        command: a Command tuple containing fields delta_p(float), the distance of the
            movement, and delta_theta(float), the rotation of the movement.
    """
    raise NotImplementedError() 

In [ ]:
# Test 3
Grader.run_single_test_inline(TestProj3, "test_03_motion_model", locals())

### <a id="exp3"></a> Experiment 3: Noisy Motion (5 points)

Now that you have implemented `motion_model`, let's play with the noise parameters. Let's first use the motion parameters of `motion_noise_covariance=np.eye(2)`:
- Initialize your particle filter using the default parameters (but pass in the above motion noise parameter and use 1000 particles), 
and step through the motion model twice. 
- Use a Command of $\Delta_p = 1m$ and $\Delta_\theta = 0$ for both times.
- Please generate plots of the $[x, y]$ component of the particles after the first and
second motions (two separate plots).

This is a pretty noisy motion! You should see the particles much more spread out than they were at the start. Why is the distribution long and skinny after one motion and then really spread out after the second motion?


<b>Submission Material 3:</b> In your submitted pdf, please include the generated figures. Name them Figure 3a and 3b.

In [ ]:
# Submission Material 3a and 3b
raise NotImplementedError()

### <a id="exp4"></a> Experiment 4: Less Noisy Motion (5 points)

Now let's try reducing the noise parameters. Please perform Experiment 3 again, but set 
`motion_noise_covariance=np.diag([1e-2, np.deg2rad(.5)**2])`.
Please generate two plots same as in experiment 3. 
The particle set should be much more compact than before and track the motion you commanded.

<b>Submission Material 4:</b> In your submitted pdf, please include the generated figures.
Name them Figure 4a and 4b.

In [ ]:
# Submission Material 4a and 4b
raise NotImplementedError()

### <a id="exp5"></a> Experiment 5: Finer Motion (5 points)

For the two experiments above, we're taking pretty big motions, commanding a meter at a time. A mobile robot would generally send commands and get measurements at a much higher frequency. For example, the robot sends commands at 10 Hz and travels at 1 m/s. Please perform an experiment that moves the robot the same 2m forward but in .1 m increments. Please upload a plot of the particles' $[x, y]$ component after 2 m.

In this question, please use the same reduced noise parameters: `motion_noise_covariance=np.diag([1e-2, np.deg2rad(.5)**2])` as in the previous experiment.

<b>Submission Material 5:</b> In your submitted pdf, please include the generated figure. Name this Figure 5.

In [ ]:
# Submission Material 5
raise NotImplementedError()

### <a id="exp6"></a> Experiment 6: Adjusting Noise for Finer Motion (5 points)

Even though all we did is changed how often we sent a command to the robot, the sample set is much more spread out. You might think that the uncertainty in the robot's position is independent of things like control frequency, but
this plot should show you that they aren't. When we adjust the control frequency, we also have to adjust the model parameters.

Let's set `motion_noise_covariance=np.diag([1e-3, np.deg2rad(.5)**2])`. 
In your submitted pdf, please include two plots of the $[x, y]$ component of the 1000 particles after 2m. 

The particle set should be much more compact again, and track the motion you commanded.

<b>Submission Material 6:</b> In your submitted pdf, please include the generated figures.
Name them Figure 6a and 6b.

In [ ]:
# Submission Material 6a and 6b
raise NotImplementedError()

### <a id="exp7"></a> Experiment 7: Unbalanced Noise (5 points)

What happens when our motion model has worse rotation noise than translation noise? 
Let's set `motion_noise_covariance=np.diag([1e-3, np.deg2rad(30)**2])`.
Please generate plots of the $[x, y]$ component of the samples after 1m, 2m and 3m of motion made in 1m increments. You should see the samples are tightly contained, but are in a curved shape. This shape is a result of the
non-linear relationship between orientation and position, and is one of the reasons we often use sampling for localization, rather than a parametric form such as a Gaussian.

<b>Submission Material 7:</b> In your submitted pdf, please include the generated figures.
Name them Figure 7a, 7b, and 7c.

In [ ]:
# Submission Material 7a, 7b, and 7c
raise NotImplementedError()

## <a id="localization_importance"></a> 2B. Localization: Computing the Importance Weights (10 points)


Let us now add in a measurement model.
Let us assume that we have a set of landmarks given by `grid_of_landmarks()` (i.e., the default set of landmarks of a Simulator).
The robot is equipped with a sensor that can measure the range and
relative bearing $[r, b]$ from the robot's location to each of these landmarks. If the
robot's pose is $(x, y, \theta)$ and the landmark position is $(l_x, l_y)$,
then range is just the Euclidean distance $$r = \sqrt{(l_x - x)^2 + (l_y
-y)^2}$$
and the bearing is the angle to the landmark in the robot's body frame (hence
relative bearing), $$b = \texttt{atan2}(l_y - y, l_x - x) - \theta.$$  Each of
these measurements is corrupted with noise $[q_r, q_b]$ distributed according
to $q_r \sim \mathcal{N}(0, \sigma^2_r)$ and $q_b \sim \mathcal{N}(0,
\sigma^2_b)$.

Please take a look at our implementation in `simulator.simulate_sensing` and
understand how the values are simulated.

Now the problem is that for a real robot, even if you know this is the model
that generates the measurements, you can't know the noise that nature applied. (That would be omniscience!)
What we can do is work out how likely each measurement is. More precisely, given a set of measurements, we can use
this model to compute the importance weights of a set of samples of the robot
pose. 

In particular, to compute the importance weight of a particle:
- Compute the _predicted measurements_ from the current state of the robot's pose and the landmark locations.
- Compute the errors $\mathit{err}$ between the predicted and true measurements.
- Compute the likelihood of the $\mathit{err}$ as the unnormalized importance weights.
The likelihood of the error is the evaluation of the probability density function (PDF) of the two-dimensional Gaussian distribution with covariance `sensor_noise_covariance` at $\mathit{err}$.
- Finally, normalize the weights so that they sum to one.

Please implement a function that takes a set of samples and measurements and
returns the importance weights of the samples.

There are a few technicalities during this process:
- We highly recommend you normalize **all the rotational values** (in radians) to the interval $[-\pi, \pi]$ (see function `normalize_angles`).
Notably, the rotational component of the $\mathit{err}$, which we denote as $\mathit{err}_\theta$, is a rotational
value in radians. Therefore, theoretically, the likelihood of $\mathit{err}_\theta$ should have been
the evaluation of the PDF of a [wrapped Gaussian distribution](https://en.wikipedia.org/wiki/Wrapped_normal_distribution).
In other words, to evaluate the likelihood of
$\mathit{err}_\theta$, we should have taken the sum of the Gaussian PDFs at the points
$\mathit{err}_\theta + 2k\pi$ for all $k \in \mathbb{Z}$.
Nonetheless, since our sensor noise is relatively small, we expect only the single point at $k=0$
$\mathit{err}_\theta \in [-\pi, \pi]$ to have a relatively large value, and all other
points to have close-to-zero values. Therefore, we may use the Gaussian distribution as a
good approximation to the true wrapped Gaussian distribution.
- The likelihood for the particles can become really small. For numerical stability,
you will want to compute the _log-likelihood_ instead of the likelihood.
To that end, you may:
    - Use the utility `multivariate_normal_logpdf` that we provided you to evaluate the log PDF.
    - Take the sum (instead of multiplications) of the log-likelihoods to compute the unnormalized log-weight of each particle.
    - Normalize the log-weights. You should use [`scipy.special.logsumexp`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.special.logsumexp.html) to evaluate
the log-sum of the weights. In short, The `logsumexp` function calculates the following expression in a more
numerically stable way:
$$
\texttt{logsumexp}(x_1, ... x_n) = \log \left( \sum_{i=1}^{n} \exp(x_i) \right).
$$ See <a href="https://gregorygundersen.com/blog/2020/02/09/log-sum-exp/" target="_blank">this blog</a> for a more detailed description of this function.
    - Use `numpy.exp` to undo the logarithm from the log-weights and return the weights.


For reference, our solution is **29** line(s) of code.

In [ ]:
@implementation_for(Localization)
def compute_weights(self: Localization,
                    measurements: Sequence[Measurement]) -> np.ndarray:
    """Compute the importance weights of the samples, based on the new
    measurement.

    Args:
        measurements: a sequence of measurements made by the robot.

    Returns:
        weights: a numpy array of importance weights, normalized to 1.
    """
    raise NotImplementedError() 

In [ ]:
# Test 4
Grader.run_single_test_inline(TestProj3, "test_04_importance_weights", locals())

## <a id="localization_pfupdate"></a> 2C. Localization: Particle Filter Update (10 points)

What we want you to do is to write a method `update` that takes in a
set of samples, the most recent command, and the most recent measurements, and then:
  * Use the motion_model method to update the sample based on the command
  * Use compute_weights to compute the sample weights from the
measurement,
  * Resample a new set of samples,
  * Blur the samples by applying noise drawn from a zero-mean Gaussian with
covariance equal to `np.diag([1e-3, 1e-3, np.deg2rad(1)**2])`. (You'll need to
recall from lecture why we do this blurring --- if you have machine learning
background, it's a form of regularization, but you don't need to know about
regularization to understand why this is important.)


For reference, our solution is **12** line(s) of code.

In addition to all the utilities defined at the top of this notebook, the following functions are available: `compute_weights`, `motion_model`. 

In [ ]:
@implementation_for(Localization)
def update(self: Localization, command: Command,
           measurements: Sequence[Measurement]) -> None:
    """Update the samples, based on the command and the new measurement.

    Args:
        command: a Command tuple containing fields delta_p(float), the distance of the
        movement, and delta_theta(float), the rotation of the movement.
        measurement: a measurement vector. The measurement is computed by
            `simulator.simulate_sensing` function.
    """
    raise NotImplementedError() 

In [ ]:
# Test 5
Grader.run_single_test_inline(TestProj3, "test_05_particle_filter", locals())

## <a id="evaluating_pf_localization"></a> Evaluating Particle Filter-based Localization (5 points)


### <a id="exp8"></a> Experiment 8: Running the Localization Particle Filter (5 points)

Let's visualize our particle filter.

Please generate a trajectory plot and an animation of the robot carrying out the 
localization inference. 
In particular:
- Use the default Simulator parameters (i.e. simply use `Simulator()`)
- Run localization with `num_particles=100`.
- Plot (and also animate) the obtained results.

If everything is working, your estimated and ground truth trajectories should match up closely, but not exactly. 
(You should be able to inspect the estimated and ground truth trajectories manually, and see that they differ by some small errors at each step.)


Here are some handy functions for your experiments:
- `run_inference`: It takes in a simulator instance and runs
inference in a particular mode --- for this question, you should pass in 
the "localization" mode. It returns a `SimulationResult` instance. 
- `plot_simulation_result`: It visualizes a `SimulationResult`.
- `render_animation`: It creates an animated visualization of a `SimulationResult`.

<b>Submission Material 8:</b> 
In your submitted pdf, please include the generated figure. Name this Figure 8.
Additionally include the generated animation. Name this 'localization.mp4'.

In [ ]:
# Submission Material 8
raise NotImplementedError()

# <a id="pf_slam"></a>3. Particle Filter SLAM (40 points)

Now, what if we don't know the landmark positions in advance?  
If the robot knew its locations exactly, it could use its observations of the landmarks to build a map of the 
landmarks' locations in absolute coordinates. But the robot is as lost as it was before --- what can we do?
This problem is known as *simultaneous localization and mapping* (SLAM) --- we need to estimate both the robot's pose *and* the map (i.e., the positions of the landmarks).

In this section, we will use the same algorithm from last section, the particle filter, to approach the SLAM problem.
Recall each particle in the localization problem only contains a robot's pose.
In the SLAM case, each particle will now contain a complete "possible world" --- made up of a robot pose and a location of each landmark that is known by to the robot so far.

### Utilities

Please have a look at the following utility functions for Particle Filter SLAM:

In [ ]:
@dataclasses.dataclass(frozen=True)
class SLAMParticles:
    """A batch of samples for SLAM."""
    x: np.ndarray  # shape: (num_particles, )
    y: np.ndarray  # shape: (num_particles, )
    theta: np.ndarray  # shape: (num_particles, )

    # Mapping from landmark_id to the index of that landmark in `landmarks_loc`
    landmarks_id_to_idx: Dict[str, int]

    landmarks_loc: np.ndarray  # shape (num_particles, num_landmarks, 2)

    def __post_init__(self):
        assert self.x.shape == self.y.shape == self.theta.shape, "shape must match"
        assert self.x.shape[0] == self.landmarks_loc.shape[
            0], "shape must match"
        assert self.landmarks_loc.shape[1] == len(
            self.landmarks_id_to_idx), "shape must match"
        object.__setattr__(self, "theta", normalize_angles(self.theta))

    def __len__(self) -> int:
        """The number of samples."""
        return self.x.shape[0]


@dataclasses.dataclass
class SLAM(Inference):

    # A 2x2 array for the motion covariance.
    motion_noise_covariance: np.ndarray

    # A 2x2 numpy array for the sensor noise for the
    # measurements of range and bearing to the landmarks
    sensor_noise_covariance: np.ndarray

    # A 2x2 array for the covariance when initializing new landmarks
    new_landmarks_init_covariance: np.ndarray = np.eye(2) * 10

    num_particles: int = 10

    particles: SLAMParticles = None

    def __post_init__(self):
        if self.particles:
            assert len(self.particles) == self.num_particles

    def estimated_pose(self) -> Pose:
        return Pose(np.mean(self.particles.x), np.mean(self.particles.y),
                    circular_mean(self.particles.theta))

    def estimated_landmarks(self) -> Sequence[Landmark]:
        landmarks_loc = np.mean(self.particles.landmarks_loc, axis=0)
        idx_to_id = {
            idx: id for id, idx in self.particles.landmarks_id_to_idx.items()
        }
        return [
            Landmark(idx_to_id[i], *loc) for i, loc in enumerate(landmarks_loc)
        ]

    def init(self, init_state: Pose) -> None:
        self.particles = SLAMParticles(
            np.full(self.num_particles, init_state.x, dtype=np.float64),
            np.full(self.num_particles, init_state.y, dtype=np.float64),
            np.full(self.num_particles, init_state.theta, dtype=np.float64),
            {},
            np.empty((self.num_particles, 0, 2), dtype=np.float64),
        )

    def get_measured_landmarks_idx(
            self, measurements: Sequence[Measurement]) -> np.ndarray:
        """Given a sequence of measurements, find the indices of each of the l
        andmark in `self.particles`.

        Args:
            measurements: a sequence of measures. This function assumes that all
            landmarks in this input sequence of measurements are already tracked
            (i.e., using `self.add_new_landmarks`).
        """
        return np.array(
            [
                self.particles.landmarks_id_to_idx[m.landmark_id]
                for m in measurements
            ],
            dtype=int,
        )

    def plot_state(self, ax: 'matplotlib.axes.Axes') -> None:
        import matplotlib.pyplot as plt
        cmap = plt.get_cmap('viridis')

        ax.quiver(
            self.particles.x,
            self.particles.y,
            np.cos(self.particles.theta),
            np.sin(self.particles.theta),
            angles="xy",
            scale=100,
            color=(1.0, 0, 0, 0.1),
        )

        idx_to_id = {
            idx: id for id, idx in self.particles.landmarks_id_to_idx.items()
        }
        landmark_idx_to_color_code = np.array([
            float(hash(idx_to_id[idx]) % 256) / 256
            for idx in range(self.particles.landmarks_loc.shape[1])
        ])

        ax.scatter(self.particles.landmarks_loc[..., 0].flatten(),
                   self.particles.landmarks_loc[..., 1].flatten(),
                   marker="x",
                   c=np.broadcast_to(
                       landmark_idx_to_color_code,
                       self.particles.landmarks_loc.shape[:-1]).flatten(),
                   cmap=cmap)

    # We will incrementally implement methods below using the `implementation_for`
    # helper
    def motion_model(self, command: Command) -> None:
        raise NotImplementedError()

    def add_new_landmarks(self, measurements: Sequence[Measurement]) -> None:
        raise NotImplementedError()

    def compute_weights(self,
                        measurements: Sequence[Measurement]) -> np.ndarray:
        raise NotImplementedError()

    def update(self, command: Command,
               measurements: Sequence[Measurement]) -> None:
        raise NotImplementedError()

## <a id="slam_motion_model"></a> 3A. SLAM: Update by Motion Model (5 points)

Please implement a function that takes in a control command $[\Delta p_t, \Delta \theta_t]$
and updates the particles by drawing from the posterior $X_{t+1}$ using the motion model.

This function behaves the same as `Localization.motion_model` for the pose
components of the particles, and simply keeps the landmarks components
(`SLAMParticles.landmarks_id_to_idx` and `SLAMParticles.landmarks_loc`)
of the particles the same.

**Hint**:
Please take a look at the fields of the `SLAMParticles` class.
Note that `landmarks_id_to_idx` stores all the landmarks the robot has encountered so far.
It is a mapping from a landmark identifier (a string) to an index (an integer).
You can use the integer index to index into the `landmarks_loc` array.
In particular, `landmarks_loc` is an array of shape `(num_particles, len(landmarks_id_to_idx), 2)`.
Therefore, to access the coordinate of the landmark with the identifier `"Stata"` contained
in the $i$-th particle:
```python
# assert isinstance(particles, SLAMParticles)
loc = particles.landmarks_loc[i, particles.landmarks_id_to_idx["Stata"]]
```


For reference, our solution is **11** line(s) of code.

In [ ]:
@implementation_for(SLAM)
def motion_model(self: SLAM, command: Command) -> None:
    """A motion model that simulates a one-step movement of the robot and updates the particles.

    Args:
        command: a Command tuple containing fields delta_p(float), the distance of the
            movement, and delta_theta(float), the rotation of the movement.
    """
    raise NotImplementedError() 

In [ ]:
# Test 6
Grader.run_single_test_inline(TestProj3, "test_06_motion_model_update", locals())

## <a id="slam_tracking"></a>3B. SLAM: Tracking New Landmarks (10 points)

Since our robot might have limited sensing range (see the parameter `Simulator.max_sensing_range`),
the robot is not aware of all the possible landmarks out there in the world.
Therefore, we have to match newly observed landmarks to ones we have seen before,
and add landmarks that we are seeing for the first time to our map.  
In some situations, the matching is difficult, but for our purposes,
we’ll assume that the landmarks are uniquely identifiable, so we know exactly
which ones we are observing on each step.

One way to handle missing data due to sensing range is to explicitly model the
sensing range in our probabilistic model of the sensors.
However, this approach would induce a difficult inference problem.  
Instead, we will take the approach of treating the missingness as uninformative.
In particular, we will consider sensor data within-range as observed variable,
and sensor data out-of-range as unobserved variables.
When we see a new landmark for the first time, we will assume that our belief
about the location of that landmark has a mean equal to that first observed location
and a standard variance (new_landmarks_init_covariance).
Note, though, that in practice it might be more common to wait until we have gotten
a few observations and use their mean as the mean of the distribution for
initializing our particles.

Please implement a function that takes a set of measurements and updates the
particles to include potentially new landmarks that are not yet tracked by the
particles.

Below we give a description for this procedure:

- Filter the input measurements to contain only measurements to the new (untracked)
landmarks.
- For each new landmark measurement and each particle:
    - Find the predicted location using the robot's pose of that particle and the landmark's measurement.
    - Add a Gaussian noise of covariance `self.new_landmarks_init_covariance` to this predicted location.
    - This noisy location becomes the landmark's location in the current particle.
- Update `self.particles.landmarks_id_to_idx` to maintain the invariance that it
holds a mapping from each tracked unique landmark identifier to the index of
that landmark in `self.particles.landmarks_loc`.

For reference, our solution is **24** line(s) of code.

In [ ]:
@implementation_for(SLAM)
def add_new_landmarks(self: SLAM, measurements: Sequence[Measurement]) -> None:
    """Add new landmarks into all particles to track them.

    Args:
        measurements: the measurements. This function will add only the new
        landmarks in these measurements (i.e., those that had not been added
        before).
    """
    raise NotImplementedError() 

In [ ]:
# Test 7
Grader.run_single_test_inline(TestProj3, "test_07_new_landmarks", locals())

## <a id="slam_importance"></a>3C. SLAM: Computing the Importance Weight (10 points)

Please implement the `SLAM.compute_weights` function that takes a
sequence of measurements and returns the importance weights of the particles.

This function is similar to `Localization.compute_weights`, except that it takes
into account the landmarks location tracked in the particles.


For reference, our solution is **17** line(s) of code.

In [ ]:
@implementation_for(SLAM)
def compute_weights(self: SLAM,
                    measurements: Sequence[Measurement]) -> np.ndarray:
    """Compute the importance weights of the particles, based on the measurement.

    Args:
        measurements: a sequence of measurements made by the robot.

    Returns:
        weights: a numpy array of importance weights, normalized to 1.
    """
    raise NotImplementedError() 

Run the following tests:

In [ ]:
# Test 8
Grader.run_single_test_inline(TestProj3, "test_08_slam_weights", locals())

## <a id="slam_pf_update"></a>3D. SLAM: Particle Filter Update (5 points)

Please implement the update step of SLAM.

The procedure is similar to `Localization.update`, except that it does an additional `add_new_landmarks`
step after forward updating with the `motion_model` and before `compute_weights`.


For reference, our solution is **16** line(s) of code.

In addition to all the utilities defined at the top of this notebook, the following functions are available in this question environment: `add_new_landmarks`, `compute_weights`, `motion_model`. You may not need to use all of them.

In [ ]:
@implementation_for(SLAM)
def update(self: SLAM, command: Command,
           measurements: Sequence[Measurement]) -> None:
    """Update the particles, based on the command and measurements.

    Args:
        command: a Command tuple containing fields delta_p(float), the distance of the
        movement, and delta_theta(float), the rotation of the movement.
        measurement: a measurement vector. The measurement is computed by
            `simulator.simulate_sensing` function.
    """
    raise NotImplementedError() 

Run the following tests:

In [ ]:
# Test 9
Grader.run_single_test_inline(TestProj3, "test_09_slam_particles", locals())

## <a id="evaluating_pf_slam"></a> Evaluating Particle Filter-based SLAM (10 points)

### <a id="exp9"></a> Experiment 9: Running the SLAM Particle Filter (5 points)

Let's visualize our SLAM particle filter.

Please generate an animation of the robot carrying out inference in the default Simulator. 

In particular:

- Run SLAM in the default Simulator with `num_particles=1000` and `max_sensing_range=6`. 
- Visualize the inference using `render_animation`.

<b>Submission Material 9:</b> 
In your submission, please include the generated animation. Name the file 'slam.mp4'.

In [ ]:
# Submission Material 9
raise NotImplementedError()

### <a id="exp10"></a> Experiment 10: More Fun with the SLAM Particle Filter (5 points)

Let's further visualize our SLAM particle filter for more settings.

Please generate plots of the robot carrying out inference in the default Simulator. 

In particular:

- Run SLAM in the default Simulator, for all combinations of these parameters:
    - `num_particles` in `[100, 1000]`
    - `max_sensing_range` in `[6, np.inf]`
- Use `plot_simulation_result` to plot the obtained estimated trajectory and landmarks for each setting. 

<b>Submission Material 10:</b> 
In your submitted pdf, please include the generated figures.
For each plot, indicate its setting (i.e., the parameter values) in the caption or title.

In [ ]:
# Submission Material 10
raise NotImplementedError()

## Final Submission
Your final submission to gradescope should include the following files:

- project02_release.ipynb: Your completed notebook with output from running each cell. Make sure to save. 
- localization.mp4: The animation of the successful run of the localizing robot. 
- slam.mp4: The animation of the successful run of the mapping robot. 
- A PDF of the results of the experiments. 

In [ ]:
# Run all tests
Grader.grade_output([TestProj3], [locals()], "results.json")
Grader.print_test_results("results.json")

In [ ]:
###############################################
#   SAVE YOUR NOTEBOOK BEFORE SUBMITTING!!!   #
###############################################

In [ ]:
# Make sure you save the notebook before running this cell so that the most updated version is zipped!
Grader.prepare_submission("Project03_release")

## Feedback

If you have any feedback for us, please complete [this form](https://forms.gle/QR5EJid3JHyCCRs66)!